# Any and all pacakges used for cleaning and modeling

In [21]:
import pandas as pd
# !pip install wrds
import wrds
import os
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
# !pip install fuzzywuzzy
from fuzzywuzzy import fuzz, process

## Part 1: Merging SPAC and IPO data
- Key Points: Self cleaned excel files with offering dates and firm names
- Both read in seperately and merged on the date variabe

In [2]:
spac_data = pd.read_csv("inputs/SPACs2016-2021.csv", header=0)
IPO_data = pd.read_csv("inputs/IPO_data.csv", header=0)


In [3]:
IPO_data['CUSIP'] = IPO_data['CUSIP'].apply(lambda x: str(x).zfill(9))
IPO_data

,Unnamed: 0,offer date,IPO name,CUSIP
0,0,01/30/1975,ROYSTER,078088610
1,1,06/09/1975,VARCO,092212610
2,2,06/10/1975,COORS ADOLPH,021701610
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410
4,4,08/26/1975,C. F. BRAUN,010564710
...,...,...,...,...
15442,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109
15443,15443,12/15/2023,Bayview Acquisition,000000nan
15444,15444,12/15/2023,ZKH GROUP LTD,98877R104
15445,15445,12/19/2023,Linkage Global Inc,G5500B102


In [4]:
IPO_data = IPO_data[IPO_data['CUSIP'] != '000000nan']
IPO_data

,Unnamed: 0,offer date,IPO name,CUSIP
0,0,01/30/1975,ROYSTER,078088610
1,1,06/09/1975,VARCO,092212610
2,2,06/10/1975,COORS ADOLPH,021701610
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410
4,4,08/26/1975,C. F. BRAUN,010564710
...,...,...,...,...
15440,15440,12/01/2023,Garden Stage Ltd,G3730L107
15441,15441,12/04/2023,Aimei Health Technology Co.,G01341117
15442,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109
15444,15444,12/15/2023,ZKH GROUP LTD,98877R104


In [5]:
spac_data['CUSIP'] = spac_data['CUSIP'].apply(lambda x: str(x).zfill(9))
spac_data

,SPAC IPO,Date of IPO,CUSIP
0,Aetherium Acquisition Corp.,12/29/2021,00809J200
1,Welsbach Technology Metals Acquisition Corp.,12/27/2021,950415208
2,Arogo Capital Acquisition Corp.,12/23/2021,042644203
3,Gardiner Healthcare Acquisitions Corp.,12/21/2021,365506203
4,Larkspur Health Acquisition Corp.,12/20/2021,51724W206
...,...,...,...
1009,CF Corp.,5/25/2016,000000nan
1010,KLR Energy Acquisition Corp.,3/16/2016,777385204
1011,Jensyn Acquisition Corp.,3/7/2016,47632B201
1012,Silver Run Acquisition Corp.,2/29/2016,82811P200


In [6]:
spac_data = spac_data[spac_data['CUSIP'] != '000000nan']
spac_data

,SPAC IPO,Date of IPO,CUSIP
0,Aetherium Acquisition Corp.,12/29/2021,00809J200
1,Welsbach Technology Metals Acquisition Corp.,12/27/2021,950415208
2,Arogo Capital Acquisition Corp.,12/23/2021,042644203
3,Gardiner Healthcare Acquisitions Corp.,12/21/2021,365506203
4,Larkspur Health Acquisition Corp.,12/20/2021,51724W206
...,...,...,...
1007,M III Acquisition Corp.,7/7/2016,55378T203
1008,"Landcadia Holdings, Inc.",6/1/2016,51476W206
1010,KLR Energy Acquisition Corp.,3/16/2016,777385204
1011,Jensyn Acquisition Corp.,3/7/2016,47632B201


In [7]:
merged_IPOnSPAC = pd.merge(left=IPO_data, right=spac_data, how='left', on='CUSIP')
merged_IPOnSPAC['SPAC IPO '] = merged_IPOnSPAC['SPAC IPO '].notna().astype(int)
merged_IPOnSPAC = merged_IPOnSPAC.drop(merged_IPOnSPAC.columns[-1], axis=1)
merged_IPOnSPAC.to_csv('inputs/MergedIPO&SPAC.csv')



In [8]:
merged_IPOnSPAC

,Unnamed: 0,offer date,IPO name,CUSIP,SPAC IPO
0,0,01/30/1975,ROYSTER,078088610,0
1,1,06/09/1975,VARCO,092212610,0
2,2,06/10/1975,COORS ADOLPH,021701610,0
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410,0
4,4,08/26/1975,C. F. BRAUN,010564710,0
...,...,...,...,...,...
15318,15440,12/01/2023,Garden Stage Ltd,G3730L107,0
15319,15441,12/04/2023,Aimei Health Technology Co.,G01341117,0
15320,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109,0
15321,15444,12/15/2023,ZKH GROUP LTD,98877R104,0


In [30]:
IPO_data = IPO_data[IPO_data['IPO name'] != 'Acquisition']
IPO_data # removing company named 'Acquisition' because it was messing with fuzzywuzzy

,Unnamed: 0,offer date,IPO name,CUSIP,MergeKey
0,0,01/30/1975,ROYSTER,078088610,ROYSTER
1,1,06/09/1975,VARCO,092212610,VARCO
2,2,06/10/1975,COORS ADOLPH,021701610,COORS
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410,KEYSTONE
4,4,08/26/1975,C. F. BRAUN,010564710,C.
...,...,...,...,...,...
15440,15440,12/01/2023,Garden Stage Ltd,G3730L107,Garden
15441,15441,12/04/2023,Aimei Health Technology Co.,G01341117,Aimei
15442,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109,INNO
15444,15444,12/15/2023,ZKH GROUP LTD,98877R104,ZKH


In [32]:
def closest_match(item, choices, matches):
    match = process.extractOne(item, choices)
    print({item, match})
    if match[0] in matches:
        return None
    matches.add(match[0])
    return match

matches = set()
spac_data['ClosestMatch'] = spac_data['SPAC IPO '].apply(lambda x: closest_match(x, IPO_data['IPO name'], matches))

spac_data

{('Aetherium Acq', 90, 15110), 'Aetherium Acquisition Corp.'}
{('Metals Acquisition Corp', 90, 14740), 'Welsbach Technology Metals Acquisition Corp.'}
{'Arogo Capital Acquisition Corp.', ('AR Capital Acquisition Corp', 95, 12564)}
{('Healthcare Acquisition', 90, 7056), 'Gardiner Healthcare Acquisitions Corp.'}
{('Larkspur Health Acquisition Co', 97, 15104), 'Larkspur Health Acquisition Corp.'}
{'Sagaliam Acquisition Corp.', ('Sagaliam Acquisition Corp', 100, 15105)}
{'NorthView Acquisition Corp.', ('NorthView Acquisition Corp', 100, 15103)}
{'Revelstone Capital Acquisition Corp.', ('Revelstone Capital Acquisition', 95, 15102)}
{('AP Acquisition Corp', 100, 15101), 'AP Acquisition Corp.'}
{('CF Acquisition Corp. VII', 100, 15096), 'CF Acquisition Corp. VII'}
{('Ault Disruptive Technologies C', 95, 15095), 'Ault Disruptive Technologies Corp.'}
{'FTAC Emerald Acquisition Corp.', ('FTAC Emerald Acquisition Corp', 100, 15097)}
{('Spree Acquisition Corp. 1 Ltd', 100, 15100), 'Spree Acquisiti

C:\Users\ziggy\AppData\Local\Temp\ipykernel_28364\3265643155.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spac_data['ClosestMatch'] = spac_data['SPAC IPO '].apply(lambda x: closest_match(x, IPO_data['IPO name'], matches))


,SPAC IPO,Date of IPO,CUSIP,MergeKey,ClosestMatch
0,Aetherium Acquisition Corp.,12/29/2021,00809J200,Aetherium,"(Aetherium Acq, 90, 15110)"
1,Welsbach Technology Metals Acquisition Corp.,12/27/2021,950415208,Welsbach,"(Metals Acquisition Corp, 90, 14740)"
2,Arogo Capital Acquisition Corp.,12/23/2021,042644203,Arogo,"(AR Capital Acquisition Corp, 95, 12564)"
3,Gardiner Healthcare Acquisitions Corp.,12/21/2021,365506203,Gardiner,"(Healthcare Acquisition, 90, 7056)"
4,Larkspur Health Acquisition Corp.,12/20/2021,51724W206,Larkspur,"(Larkspur Health Acquisition Co, 97, 15104)"
...,...,...,...,...,...
1007,M III Acquisition Corp.,7/7/2016,55378T203,M,"(M III Acquisition Corp, 100, 12872)"
1008,"Landcadia Holdings, Inc.",6/1/2016,51476W206,Landcadia,None
1010,KLR Energy Acquisition Corp.,3/16/2016,777385204,KLR,None
1011,Jensyn Acquisition Corp.,3/7/2016,47632B201,Jensyn,"(Jensyn Acquisition Corp, 100, 12832)"


In [93]:
# IPO_data.rename(columns={'offer date' : 'Date of IPO'}, inplace=True)
# spac_data['MergeKey'] = spac_data['ClosestMatch'].apply(lambda x: x[0])
# test_merge = pd.merge(left=IPO_data, right=spac_data, how='left', left_on='IPO name', right_on='MergeKey')
# test_merge#['ClosestMatch'].isna().sum()



spac_data.to_csv('inputs/SPACReadyForMerge.csv')

# spac_data = spac_data.reset_index()

# spac_data
# for row in range(len(spac_data)):
#     if(spac_data['ClosestMatch'][row][0] != ''):
#         spac_data.loc[row, 'MergeKey'] = spac_data['ClosestMatch'][row][0]

for row in range(len(spac_data)):
    closest_match = spac_data.at[row, 'ClosestMatch']
    if closest_match is not None and closest_match[0] != '':
        spac_data.at[row, 'MergeKey'] = closest_match[0]

spac_data
# for row in range(len(spac_data)):
#     closest_match = spac_data['ClosestMatch'][row]
#     if closest_match and len(closest_match) > 0 and closest_match[0] is not None:
#         spac_data.loc[row, 'MergeKey'] = closest_match[0] if (closest_match[0] != '') else None; 


# spac_data

,level_0,index,SPAC IPO,Date of IPO,CUSIP,MergeKey,ClosestMatch
0,0,0,Aetherium Acquisition Corp.,12/29/2021,00809J200,Aetherium Acq,"(Aetherium Acq, 90, 15110)"
1,1,1,Welsbach Technology Metals Acquisition Corp.,12/27/2021,950415208,Metals Acquisition Corp,"(Metals Acquisition Corp, 90, 14740)"
2,2,2,Arogo Capital Acquisition Corp.,12/23/2021,042644203,AR Capital Acquisition Corp,"(AR Capital Acquisition Corp, 95, 12564)"
3,3,3,Gardiner Healthcare Acquisitions Corp.,12/21/2021,365506203,Healthcare Acquisition,"(Healthcare Acquisition, 90, 7056)"
4,4,4,Larkspur Health Acquisition Corp.,12/20/2021,51724W206,Larkspur Health Acquisition Co,"(Larkspur Health Acquisition Co, 97, 15104)"
...,...,...,...,...,...,...,...
876,876,1007,M III Acquisition Corp.,7/7/2016,55378T203,M III Acquisition Corp,"(M III Acquisition Corp, 100, 12872)"
877,877,1008,"Landcadia Holdings, Inc.",6/1/2016,51476W206,Battery Future Acquisition Cor,None
878,878,1010,KLR Energy Acquisition Corp.,3/16/2016,777385204,Battery Future Acquisition Cor,None
879,879,1011,Jensyn Acquisition Corp.,3/7/2016,47632B201,Jensyn Acquisition Corp,"(Jensyn Acquisition Corp, 100, 12832)"


In [96]:
SPAC_IPO_merge = pd.merge(left= IPO_data, right=spac_data, how='left', left_on='IPO name', right_on='MergeKey')
SPAC_IPO_merge

,Unnamed: 0,offer date,IPO name,CUSIP_x,MergeKey_x,level_0,index,SPAC IPO,Date of IPO,CUSIP_y,MergeKey_y,ClosestMatch
0,0,01/30/1975,ROYSTER,078088610,ROYSTER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,06/09/1975,VARCO,092212610,VARCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,06/10/1975,COORS ADOLPH,021701610,COORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410,KEYSTONE,55.0,56.0,Vahanna Tech Edge Acquisition I Corp.,11/23/2021,G9320Z125,KEYSTONE FOODS CORP,"(KEYSTONE FOODS CORP, 86, 3)"
4,4,08/26/1975,C. F. BRAUN,010564710,C.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15495,15440,12/01/2023,Garden Stage Ltd,G3730L107,Garden,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15496,15441,12/04/2023,Aimei Health Technology Co.,G01341117,Aimei,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15497,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109,INNO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15498,15444,12/15/2023,ZKH GROUP LTD,98877R104,ZKH,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
#Reading in ccm data

ccm_data = pd.read_csv('inputs/cleaned_ccm.csv')
ccm_data

,Unnamed: 0,datadate,cusip,scf,src,apdedate,fdate,pdate,acchg,acominc,...,seq,teq,tstk,txt,wcap,naicsh,mkvalt,Seq. No.,Code,Industry
0,20283,2014-12-31,000307108,7.0,5.0,2014-12-31,2015-04-13,2015-02-25,0.000,0.000,...,97.474,95.141,0.0,2.555,63.153,622210.0,660.8841,1821.0,622210.0,Psychiatric and Substance Abuse Hospitals
1,23764,2001-12-31,000375204,7.0,53.0,NaN,NaN,NaN,-63.000,-1699.000,...,2014.000,NaN,1750.0,105.000,479.000,335311.0,NaN,790.0,335311.0,"Power, Distribution, and Specialty Transformer..."
2,795,2014-12-31,000899104,7.0,5.0,2014-12-31,2015-04-10,2015-03-11,0.000,0.000,...,6.009,6.009,0.0,-0.552,19.890,325414.0,103.7916,506.0,325414.0,Biological Product (except Diagnostic) Manufac...
3,5229,2001-04-30,00089C107,7.0,5.0,NaN,NaN,NaN,-1.785,1.413,...,104.664,NaN,0.0,0.037,71.958,334515.0,216.1704,770.0,334515.0,Instrument Manufacturing for Measuring and Tes...
4,5683,2000-12-31,00089J102,7.0,5.0,NaN,NaN,NaN,0.000,NaN,...,4.468,NaN,0.0,0.065,3.768,334413.0,7.9185,758.0,334413.0,Semiconductor and Related Device Manufacturing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5497,20504,2007-12-31,Y8565N300,7.0,5.0,2007-12-31,2008-05-23,NaN,0.000,0.912,...,148.794,148.794,0.0,0.000,32.806,483111.0,550.0000,1220.0,483111.0,Deep Sea Freight Transportation
5498,14415,2004-12-31,Y8897Y230,7.0,8.0,2004-12-31,NaN,NaN,0.000,-0.248,...,321.809,NaN,0.0,0.000,98.240,483111.0,452.2538,1220.0,483111.0,Deep Sea Freight Transportation
5499,2671,2014-12-31,Y8977Y100,7.0,5.0,2014-12-31,2015-04-02,2015-02-26,0.000,0.000,...,1411.000,2451.000,0.0,20.000,143.000,213111.0,1008.2829,168.0,213111.0,Drilling Oil and Gas Wells
5500,2632,2014-12-31,Y9384M101,7.0,5.0,2014-12-31,2015-05-13,NaN,0.000,-6.000,...,283.100,835.200,0.0,16.300,46.100,424710.0,997.7975,1037.0,424710.0,Petroleum Bulk Stations and Terminals


In [100]:

SPAC_IPO_merge.rename(columns={'CUSIP_x' : 'cusip'}, inplace=True)
merge_df_ccm = pd.merge(left=SPAC_IPO_merge, right=ccm_data, on='cusip', how='inner')

merge_df_ccm

,Unnamed: 0_x,offer date,IPO name,cusip,MergeKey_x,level_0,index,SPAC IPO,Date of IPO,CUSIP_y,...,seq,teq,tstk,txt,wcap,naicsh,mkvalt,Seq. No.,Code,Industry
0,2089,06/05/1986,Genzyme,372917104,Genzyme,NaN,NaN,NaN,NaN,NaN,...,1750.280,NaN,0.901,57.140,438.733,325414.0,8587.5923,506.0,325414.0,Biological Product (except Diagnostic) Manufac...
1,4619,06/16/1992,Columbia Banking System,197236102,Columbia,NaN,NaN,NaN,NaN,NaN,...,78.800,NaN,0.000,6.122,NaN,522110.0,122.3405,1420.0,522110.0,Commercial Banking
2,4989,02/19/1993,Nuveen Prem Inc Muni Fund 4,6706K4105,Nuveen,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,525990.0,NaN,1490.0,525990.0,Other Financial Vehicles
3,5106,04/22/1993,TCW/DW Term Trust 2003,87234U108,TCW/DW,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,525990.0,NaN,1490.0,525990.0,Other Financial Vehicles
4,5158,05/20/1993,Nuveen CT Prem Inc Muni Fund,67060D107,Nuveen,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,525990.0,NaN,1490.0,525990.0,Other Financial Vehicles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,13368,11/15/2018,Bain Capital Specialty Finance,05684B107,Bain,NaN,NaN,NaN,NaN,NaN,...,1001.629,1001.629,0.000,0.000,NaN,523910.0,863.3531,1452.0,523910.0,Miscellaneous Intermediation
2138,13376,12/07/2018,Moderna Inc,60770K107,Moderna,NaN,NaN,NaN,NaN,NaN,...,1530.241,1530.241,0.000,0.326,1340.203,325414.0,5020.7607,506.0,325414.0,Biological Product (except Diagnostic) Manufac...
2139,13377,12/07/2018,Synthorx Inc,87167A103,Synthorx,NaN,NaN,NaN,NaN,NaN,...,184.360,184.360,0.000,0.000,183.002,325414.0,545.6451,506.0,325414.0,Biological Product (except Diagnostic) Manufac...
2140,13381,12/14/2018,360 Fin Inc,88557W101,360,NaN,NaN,NaN,NaN,NaN,...,645.581,645.581,0.000,67.806,646.750,522220.0,NaN,1429.0,522220.0,Sales Financing


# Filtering Desired Variables

In [101]:
relevant_vars = ['SPAC IPO ','offer date','IPO name','adrr','curuscn','scf','src','apdedate','fdate','pdate','acominc',
                 'acox','at','am','ao','aoloch','aox','ap','at','caps','capx','cb',
                 'ch','che','clg','cogs','csho','cusip','cshrt','cstk','dd','dlc','dn','do',
                 'datadate','dt','ebit','ebitda','epspi','fca','ffo','gdwl','gp','ib','intan',
                 'invt','lt','lct','ni','niadj','np','pi','ppegt','pnrsho','ppent',
                 're','revt','sale','seq','tdc','teq','tstk','txt','wcap','naicsh',
                 'mkvalt','acchg','accrt','amc','ano','arce','cshi','depc','derhedgl']

cleaned_data = merge_df_ccm[[col for col in merge_df_ccm.columns if col in relevant_vars]]
cleaned_data.to_csv("inputs/cleaned_data.csv")

Beginning Modeling

In [107]:
KNN_data = pd.read_csv("inputs/cleaned_data.csv")
KNN_data

,Unnamed: 0,offer date,IPO name,cusip,SPAC IPO,datadate,scf,src,apdedate,fdate,...,re,revt,sale,seq,teq,tstk,txt,wcap,naicsh,mkvalt
0,0,06/05/1986,Genzyme,372917104,NaN,2000-12-31,7.0,5.0,NaN,NaN,...,480.941,752.483,752.483,1750.280,NaN,0.901,57.140,438.733,325414.0,8587.5923
1,1,06/16/1992,Columbia Banking System,197236102,NaN,2000-12-31,NaN,53.0,NaN,NaN,...,34.181,67.914,67.914,78.800,NaN,0.000,6.122,NaN,522110.0,122.3405
2,2,02/19/1993,Nuveen Prem Inc Muni Fund 4,6706K4105,NaN,2000-10-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,525990.0,NaN
3,3,04/22/1993,TCW/DW Term Trust 2003,87234U108,NaN,2001-03-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,525990.0,NaN
4,4,05/20/1993,Nuveen CT Prem Inc Muni Fund,67060D107,NaN,2001-05-31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,525990.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,2137,11/15/2018,Bain Capital Specialty Finance,05684B107,NaN,2018-12-31,7.0,5.0,2018-12-31,2019-03-27,...,-32.678,99.294,99.294,1001.629,1001.629,0.000,0.000,NaN,523910.0,863.3531
2138,2138,12/07/2018,Moderna Inc,60770K107,NaN,2018-12-31,7.0,5.0,2018-12-31,2019-04-09,...,-1007.947,135.068,135.068,1530.241,1530.241,0.000,0.326,1340.203,325414.0,5020.7607
2139,2139,12/07/2018,Synthorx Inc,87167A103,NaN,2018-12-31,7.0,5.0,2018-12-31,2019-04-09,...,-69.478,0.000,0.000,184.360,184.360,0.000,0.000,183.002,325414.0,545.6451
2140,2140,12/14/2018,360 Fin Inc,88557W101,NaN,2018-12-31,7.0,5.0,2018-12-31,2019-06-26,...,-62.023,646.572,646.572,645.581,645.581,0.000,67.806,646.750,522220.0,NaN


## KNN Modeling 

In [108]:
data_types = pd.DataFrame(KNN_data.dtypes)
#data_types.to_csv("inputs/data_types.csv")

In [109]:

date_cols = ['offer date', 'datadate', 'apdedate', 'fdate', 'pdate']
for col in date_cols:
    KNN_data[col] = pd.to_datetime(KNN_data[col], errors='coerce')

# Exclude non-numeric columns properly
exclude_cols = ['IPO name', 'cusip', 'offer date', 'datadate', 'apdedate', 'fdate', 'pdate']  # Including date columns to exclude
numeric_cols = KNN_data.columns.difference(exclude_cols + ['IPO name', 'cusip', 'SPAC IPO '])

# Convert to numeric and handle missing values only for numeric columns
KNN_data[numeric_cols] = KNN_data[numeric_cols].apply(pd.to_numeric, errors='coerce')
KNN_data[numeric_cols] = KNN_data[numeric_cols].fillna(KNN_data[numeric_cols].median())

In [110]:
null_df = KNN_data.isnull().sum()
print(len(null_df))

KNN_data = KNN_data.dropna()

61


In [111]:
import numpy as np

# X = KNN_data[numeric_cols]  # Removed the extra space in the column name
# y = KNN_data['SPAC IPO '].dropna()  # Ensure no extra spaces here

rng = np.random.RandomState(0)
# Split data
X_train, X_test, y_train, y_test = train_test_split(KNN_data[numeric_cols], KNN_data['SPAC IPO '].dropna(), test_size=0.3, random_state=rng)

#Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Evaluate the model
predictions = knn.predict(X_test_scaled )
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [112]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

feature_names = numeric_cols[:2]  # Replace with actual names if known
X_plot = KNN_data[feature_names]

# Scaling
scaler = StandardScaler()
X_plot_scaled = scaler.fit_transform(X_plot)

# Split data
X_train_plot, X_test_plot, y_train_plot, y_test_plot = train_test_split(X_plot_scaled, KNN_data['SPAC IPO '].dropna(), test_size=0.3, random_state=rng)

# Train KNN
knn_plot = KNeighborsClassifier(n_neighbors=5)
knn_plot.fit(X_train_plot, y_train_plot)

# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#0000FF'])

# Plot the decision boundary
x_min, x_max = X_train_plot[:, 0].min() - 1, X_train_plot[:, 0].max() + 1
y_min, y_max = X_train_plot[:, 1].min() - 1, X_train_plot[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
Z = knn_plot.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(10, 6))
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# Plot also the training points
plt.scatter(X_train_plot[:, 0], X_train_plot[:, 1], c=y_train_plot, cmap=cmap_bold, edgecolor='k', s=20)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("2-Class classification (k = 5, weights = 'uniform')")
plt.xlabel(feature_names[0])
plt.ylabel(feature_names[1])

plt.show()

ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required by StandardScaler.